In [1]:
import sys
import re
import datetime
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from pyspark import *
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType, ArrayType, StructType, StructField, FloatType, DoubleType
from pyspark.ml.feature import VectorAssembler, StringIndexer,  OneHotEncoder, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import LinearRegression,  RandomForestRegressor
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import SparseVector, DenseVector

In [2]:
sc.stop()
spark = SparkSession.builder.appName('regression').getOrCreate()

In [3]:
data = spark.read.format('csv').option('header', 'true').option('inferSchema','true').load('data_cleaned.csv')
data = data.limit(100000)                                                                       

In [4]:
data = data.repartition(500)

In [5]:
data_processed = data.withColumn('status', data.status.cast(StringType())).withColumn(
    'label', data.signal.cast(FloatType())).drop('signal','provider')
data_processed.show()

+--------+--------+--------+------+---+-----+----------+----------+----------+-------+-----+
|     lat|    long|operator|status|net|speed|satellites|precission|  activity|hour_24|label|
+--------+--------+--------+------+---+-----+----------+----------+----------+-------+-----+
|41.44828| 2.19933|  orange|     2| 4G| 11.1|      12.0|      16.0|IN_VEHICLE|     14| 14.0|
|41.69224| 1.78829|movistar|     2| 4G|  4.9|      11.0|       7.0|   ON_FOOT|     12|  1.0|
|41.41603|  2.1878|movistar|     0| 4G| 13.2|       6.0|      22.0|   ON_FOOT|     12| 20.0|
|41.48979| 2.16727|  orange|     2| 4G| 10.4|       7.0|      14.0|ON_BICYCLE|     11| 18.0|
|41.43914| 1.86436| jazztel|     2| 2G| 13.3|       5.0|      32.0|IN_VEHICLE|     22|  5.0|
|26.68327|-81.9076|  others|     2| 4G|  1.9|       8.0|       3.0|   ON_FOOT|     17| 20.0|
| 41.9066| 2.24868| jazztel|     2| 3G|113.8|      11.0|      15.0|IN_VEHICLE|     18| 12.0|
|42.14808| 2.57902|  orange|     0| 2G|  2.5|       5.0|       9.0|IN_

In [6]:
data_processed.dtypes

[('lat', 'double'),
 ('long', 'double'),
 ('operator', 'string'),
 ('status', 'string'),
 ('net', 'string'),
 ('speed', 'double'),
 ('satellites', 'double'),
 ('precission', 'double'),
 ('activity', 'string'),
 ('hour_24', 'int'),
 ('label', 'float')]

In [7]:
categoricalCols = [field for (field, dataType) in data_processed.dtypes if dataType == "string"]
inputOutputCols = [x+"index" for x in categoricalCols]
oheOutputCols = [x+"OHE" for x in categoricalCols]
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=inputOutputCols,
                             handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=inputOutputCols,
                          outputCols=oheOutputCols)
data_indexed = stringIndexer.fit(data_processed).transform(data_processed)
df = oheEncoder.fit(data_indexed).transform(data_indexed)
df.show()

+--------+--------+--------+------+---+-----+----------+----------+----------+-------+-----+-------------+-----------+--------+-------------+-------------+-------------+-------------+-------------+
|     lat|    long|operator|status|net|speed|satellites|precission|  activity|hour_24|label|operatorindex|statusindex|netindex|activityindex|  operatorOHE|    statusOHE|       netOHE|  activityOHE|
+--------+--------+--------+------+---+-----+----------+----------+----------+-------+-----+-------------+-----------+--------+-------------+-------------+-------------+-------------+-------------+
|41.44828| 2.19933|  orange|     2| 4G| 11.1|      12.0|      16.0|IN_VEHICLE|     14| 14.0|          1.0|        0.0|     0.0|          0.0|(6,[1],[1.0])|(3,[0],[1.0])|(2,[0],[1.0])|(4,[0],[1.0])|
|41.69224| 1.78829|movistar|     2| 4G|  4.9|      11.0|       7.0|   ON_FOOT|     12|  1.0|          0.0|        0.0|     0.0|          1.0|(6,[0],[1.0])|(3,[0],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|
|41.41603|

In [8]:
numeric_cols = [field for (field, dataType) in data_processed.dtypes if ((dataType != "string") & (field !='label'))]
assembled_inputs = numeric_cols+oheOutputCols
assembled_inputs
vecAssembler = VectorAssembler(inputCols=assembled_inputs, outputCol='features')
train_df, test_df = vecAssembler.transform(df).randomSplit([0.8, 0.2], seed=777)
#train_df.withColumn('features', DenseVector(train_df.features)).show()
test_df.select('features').show()

+--------------------+
|            features|
+--------------------+
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,6,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,3,4,5,7,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
|(21,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [9]:
scala = StandardScaler(inputCol='features',  outputCol='scaled_features', withStd=True, withMean=True)
scala.fit(test_df).transform(test_df).select(F.col('scaled_features')[0]).take(2)

AnalysisException: Can't extract value from scaled_features#4473: need struct type but got struct<type:tinyint,size:int,indices:array<int>,values:array<double>>

In [ ]:
rf = RandomForestRegressor(featuresCol='features', labelCol='label')
rf_model = rf.fit(train_df)
predicted_df = rf_model.transform(test_df)
evaluator = RegressionEvaluator(labelCol='label', predictionCol='prediction')
rmse = evaluator.evaluate(rf_model.transform(test_df), {evaluator.metricName: "rmse"})
rmse

In [ ]:
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [35, 33, 37]) \
    .addGrid(rf.maxDepth, [10]) \
    .build()
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
tvs = TrainValidationSplit(estimator=rf,
                           estimatorParamMaps=param_grid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)
tvs_model = tvs.fit(train_df)

In [ ]:
rmse = evaluator.evaluate(tvs_model.transform(test_df), {evaluator.metricName: "rmse"})
rmse

In [ ]:
tvs_model.bestModel.extractParamMap()

In [ ]:
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()
cv = CrossValidator(estimator=rf, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(),
                    numFolds=3)
cvmodel = cv.fit(train_df)

In [ ]:
cvmodel.transform(test_df)

In [ ]:
evaluator = RegressionEvaluator(labelCol='label', predictionCol='prediction')
rmse = evaluator.evaluate(cvmodel.transform(test_df), {evaluator.metricName: "rmse"})
rmse

In [ ]:
rmse = evaluator.evaluate(rf_model.transform(test_df), {evaluator.metricName: "rmse"})
rmse